# Social Media Relationships with Ordinary Least Squares

**Linear models** in the form of **ordinary least squares regression** can often be help with understanding relationships between variables. In this exercise you will analyze synthetic data which relates game players' characteristics to social media engagement. 

To start, execute the code in the cell below to import the packages you will need for this example. 

In [1]:
import numpy as np
import numpy.random as nr
from scipy.stats import truncnorm
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

## matplotlib with display of graphs inline
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

The code in the cell below generates synthetic data with characteristics of game players and social media engagement. These variables are drawn from a truncated multivariate Normal distribution, with limits of 0.0 and 10.0. Execute this code.

In [2]:
## Create data set as multivariate Normal
covariance = np.array([[1.0,0.7,0.7,0.0],
                      [0.7,1.0,0.6,0.7],
                      [0.7,0.6,1.0,0.1],
                      [0.0,0.7,0.1,1.0]])
effect_data = pd.DataFrame(nr.multivariate_normal(mean=[3.0,2.0,2.0,2.0], cov=covariance, size=500),
                           columns=['Fan','TimePlaying','SocialMedia','GameFamiliarity'])

## Truncate values to range 0.0 <= x <= 10.0
effect_data[effect_data < 0.0] = 0.0
effect_data[effect_data > 10.0] = 10.0

The code in the cell below transforms the TimePlaying variable by squaring the values. This transformation gives the transformed variable a positive skew. The code also rounds the values to two digits. Execute this code and examine the resulting data frame. 

In [3]:
## Square the values of TimePlaying to give positive skew
effect_data['TimePlaying'] = np.square(effect_data['TimePlaying'])

## Standardize the dependent variables
dependent_cols = ['Fan','TimePlaying','GameFamiliarity']
for col in dependent_cols: 
    effect_data[col] = (effect_data[col] - effect_data[col].mean()).divide(effect_data[col].var())

## And round all values to 2 decimal places
effect_data = effect_data.round(decimals=2)
effect_data

,Fan,TimePlaying,SocialMedia,GameFamiliarity
0,0.84,0.18,1.86,0.31
1,-1.02,-0.14,0.10,0.71
2,0.49,0.02,2.53,-0.20
3,0.10,0.48,2.20,2.17
4,-0.19,-0.25,1.34,-1.18
5,0.14,0.10,2.96,0.39
6,0.91,0.10,2.44,-0.50
7,0.68,-0.10,2.19,-0.69
8,-0.54,-0.02,2.11,0.50
9,-0.10,-0.28,0.00,-1.47


The code in the cell below finds a linear models SocialMedia engagement as a function of TimePlaying. Execute this code and examine the results.   

> The code in the cell below uses the R style model formula. This modeling language was introduced in Chambers and Hastie, 1992, Statistical Models in S.     

> For a good [**cheatsheet and summary of the R modeling language**](http://faculty.chicagobooth.edu/richard.hahn/teaching/formulanotation.pdf) look at the posting by Richard Hahn of the Chicago Booth School.    

> Models are defined by an equation using the $\sim$ symbol to mean modeled by. In summary, the variable to be modeled is always on the left. The relationship between the variable to be modeled on the right. This basic scheme can be written: 

$$dependent\ variable\sim indepenent\ variables$$

> For example, if the dependent variable (dv) is modeled by two independent variables (var1 and var2), with no interaction, the formula would be:
$$dv \sim var1 + var2$$

In [4]:
social_time_model = smf.ols(formula = 'SocialMedia ~ TimePlaying', data=effect_data).fit()
social_time_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            SocialMedia   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     197.7
Date:                Wed, 08 Jan 2020   Prob (F-statistic):           4.75e-38
Time:                        11:58:37   Log-Likelihood:                -612.44
No. Observations:                 500   AIC:                             1229.
Df Residuals:                     498   BIC:                             1237.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.0120      0.037     54.515      0.000       1.939       2.085
TimePlaying     2.0664      0.147     14.059      0.000       1.778       2.355
==============================================================================
Omnibus:                        6.224   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.045   Jarque-Bera (JB):                4.866
Skew:                           0.136   Prob(JB):                       0.0878
Kurtosis:                       2.601   Cond. No.                         3.98
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The summary of the ordinary least squares model contains quite a lot information. We will only focus on a few of these values:    
- The value of the coefficient is an estimate of the **effect size**. In this case the effect being measured is TimePlaying. 
- The confidence interval of the coefficient value which indicates the range of likely values for the effect size. It is important to keep in mind that any estimated effect size is uncertain and not exact. 
- The adjusted R-squared value which is the ratio of the ratio of the variance of the residuals of from the fitted model and the variance of the dependent variable. 



> #### Summary of R-Squared

> - **R squared or $R^2$**, also known as the **coefficient of determination**,  
$$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$$  
where,   
$SS_{res} = \sum_{i=1}^N r_i^2$, or the sum of the squared residuals,   
$SS_{tot} = \sum_{i=1}^N y_i^2$, or the sum of the squared label values.  

> In other words, $R^2$ is  measure of the reduction in sum of squared values between the raw label values and the residuals. If the model has not reduced the sum of squares of the labels (a useless model!), $R^2 = 0$. On the other hand, if the model fits the data perfectly so all $r_i = 0$, then $R^2 = 1$. 

> - **Adjusted R squared or $R^2_{adj}$** is $R^2$ adjusted for degrees of freedom in the model,
$$R^2_{adj} = 1 - \frac{var(r)}{var(y)} = 1 - \frac{\frac{SS_{res}}{(n - p -1)}}{\frac{SS_{tot}}{(n-1)}}$$  
where,   
$var(r) = $ the variance of the residuals,   
$var(y) = $ the variance of the labels,
$n = $ the number of samples or cases,
$p = $ number of model parameters.  

> The interpretation of $R^2_{adj}$ is the same as $R^2$. In many cases there will be little difference. However if the number of parameters is significant with respect to the number of cases, $R^2$ will give an overly optimistic measure of model performance. In general, the difference between $R^2_{adj}$ and $R^2$ becomes less significant as the number of cases $n$ grows. However, even for 'big data' models there can be a significant difference if there are a large number of model parameters.   


As a next step in the analysis we will add another variable to the model, Fan. Execute the code in the cell below and examine the summary of the new model. 

In [5]:
social_time_fan_model = smf.ols(formula='SocialMedia ~ TimePlaying + Fan', data=effect_data).fit()
social_time_fan_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            SocialMedia   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     197.8
Date:                Wed, 08 Jan 2020   Prob (F-statistic):           6.53e-64
Time:                        12:10:40   Log-Likelihood:                -549.64
No. Observations:                 500   AIC:                             1105.
Df Residuals:                     497   BIC:                             1118.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.0124      0.033     61.760      0.000       1.948       2.076
TimePlaying     0.8378      0.166      5.054      0.000       0.512       1.163
Fan             0.4969      0.042     11.913      0.000       0.415       0.579
==============================================================================
Omnibus:                        0.326   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.850   Jarque-Bera (JB):                0.440
Skew:                           0.026   Prob(JB):                        0.802
Kurtosis:                       2.864   Cond. No.                         5.21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Notice two changes between the first model and the model using two variables:
- The coefficient (effect) value for the variable Fan is marginally larger than the effect for TimePlaying. 
- The adjusted R-squared value is larger indicating this model explains more of the variance of the data.    

Another possibility is to model social media engagement by TimePlaying and GameFamiliarity. Execute the code in the cell below, examine the results, and compare them to the previous models. 

In [6]:
social_time_familarity_model = smf.ols(formula='SocialMedia ~ TimePlaying + GameFamiliarity', data=effect_data).fit()
social_time_familarity_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            SocialMedia   R-squared:                       0.437
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                     193.0
Date:                Wed, 08 Jan 2020   Prob (F-statistic):           9.36e-63
Time:                        12:22:17   Log-Likelihood:                -552.32
No. Observations:                 500   AIC:                             1111.
Df Residuals:                     497   BIC:                             1123.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           2.0117      0.033     61.411      0.000       1.947       2.076
TimePlaying         3.3259      0.170     19.612      0.000       2.993       3.659
GameFamiliarity    -0.4853      0.042    -11.624      0.000      -0.567      -0.403
==============================================================================
Omnibus:                        1.034   Durbin-Watson:                   2.056
Prob(Omnibus):                  0.596   Jarque-Bera (JB):                1.112
Skew:                          -0.066   Prob(JB):                        0.573
Kurtosis:                       2.811   Cond. No.                         5.42
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Notice that the sign of the GameFamiliarity effect is now negative. Given these results, we say that GameFamiliarity is a **confounding effect**. In other words, adding GameFamiliarity confounds, or masks the other effects, but is not explanitory. 

Given the above analysis, it seems that Fan is the variable that best explains social media engagement. This idea is easy to test, by creating a linear model of social media engagement as a function of Fan. Execute the code in the cell below to compute the model and display the summary. 

In [7]:
social_fan_model = smf.ols(formula='SocialMedia ~ Fan', data=effect_data).fit()
social_fan_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            SocialMedia   R-squared:                       0.415
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     352.6
Date:                Wed, 08 Jan 2020   Prob (F-statistic):           7.11e-60
Time:                        12:25:33   Log-Likelihood:                -562.17
No. Observations:                 500   AIC:                             1128.
Df Residuals:                     498   BIC:                             1137.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0125      0.033     60.297      0.000       1.947       2.078
Fan            0.6281      0.033     18.778      0.000       0.562       0.694
==============================================================================
Omnibus:                        0.014   Durbin-Watson:                   2.038
Prob(Omnibus):                  0.993   Jarque-Bera (JB):                0.068
Skew:                          -0.003   Prob(JB):                        0.966
Kurtosis:                       2.943   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Notice the following points about this model:
- The effect size for Fan is large relative to effect sizes in other models.  
- The R-squared value is nearly as large as the best previous model, indicating that the Fan variable has good explanatory power. 
- The confidence interval of the effect size is relatively narrow.  

From this analysis, what can you conclude about the relationship between Fan and TimePlaying and SocialMedia? Keep in mind that we will generally prefer simpler explanatory models.  

##### Copyright 2020, Stephen F Elston. All rights reserved. 